# What this dataset is all about ?
* comodities news articles from reuters , bloomberg website scrapped and other news blogs for financial news 
* each news has sentiment values from 2 to 10 
* for confidiential reason dataset can't be done public
* Task was to build financial sentiment classifier which can beat bloomberg company model so our baseline mark is 80% (bloomberg model) we have to cross this line 

## Importing Libraries for necessary visualization of dataset and gensim for creating summary

In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn
from gensim.summarization.summarizer import summarize

## These are datasets in which webscrapped data is there

In [ ]:
df1=pd.read_csv('../input/commodities/commodities_feb_1k.csv',encoding="latin-1")
df2=pd.read_csv('../input/commodities/commodities_feb_3.6k.csv',encoding="latin-1")
df3=pd.read_csv('../input/commodities/commodities_feb_4.9k.csv',encoding="latin-1")
df4=pd.read_csv('../input/commodities/commodities_jan_10k.csv',encoding="latin-1")
df5=pd.read_csv('../input/commodities/commodities_jan_5k.csv',encoding="latin-1")
df6=pd.read_csv('../input/commodities/commodities_jan_2k.csv',encoding="latin-1")


## Concatenation and reshuffling the concatenated data

In [ ]:
df4=pd.concat([df1,df2,df3,df5,df6,df4])

In [ ]:
final_df= df4.sample(frac=1).reset_index(drop=True)

## gensim needed for summarization 

In [ ]:
from gensim.summarization.textcleaner import split_sentences

## appostohphe list used for replacing the text , but  i did not  use it here for now

In [ ]:
#appos list 
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [ ]:
final_df=final_df.dropna()

In [ ]:
final_df.isnull().sum()

## basic pre-processing start down 1st spliting of text 

In [ ]:
final_df['Text'] = final_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
final_df['Text'].head()

## removal of punctuations 

In [ ]:
# removing punctuation
final_df['Text']=final_df['Text'].str.replace('[^\w\s]','')

## removing of stopwords

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
final_df['Text']=final_df['Text'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

## lemmatize is more effective than stemming using textblob 
## why lemmatize because , after lemmatize we will get ground value word rather than just if we do stemming is not so 

## Stemming and Lemmatization both generate the root form of the inflected words. The difference is that stem might not be an actual word whereas, lemma is an actual language word.

## Stemming follows an algorithm with steps to perform on the words which makes it faster. Whereas, in lemmatization, you used WordNet corpus and a corpus for stop words as well to produce lemma which makes it slower than stemming. You also had to define a parts-of-speech to obtain the correct lemma.

In [ ]:
from textblob import Word
final_df['Text'] = final_df['Text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# basic data cleaning is done , the tokenization and padding can be done with keras before making deep learning models 

# Checking distribution of sentiment 

In [ ]:
sns.countplot(final_df.Sentiment)
plt.show()

## From this we can see easily that there is distribution differences so we have to do binning of sentiment categories in 2 or 3 parts like 1 , 0 ,-1 or just 1 , 0 
## 1 is positive and 0 is negative 

In [ ]:
sns.distplot(final_df.Sentiment)
plt.show()

## we can club upto three categories because as we can that 0 and 1 is very less and 9 and 10 to also very less 
## so we have idea to do binding upto three classes low , med and high 

## for low we can do binding from 2 to 4 level sentiment 
## for medium we can do binding from 5 to 7 level sentiment 
## for high we can do binding from 7 to 10 level sentiment 

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
).generate(str(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(final_df["Text"])

## in case of low relvance the most of the article with sentiment are around 2 to 5 

## So we have 304 rows with text from which full text has not been properly came up , if models does not work well , we will see this into our account 

## Most of the data cleaning part has been done and visualization has been also done , insight was bring down that we have to binding to 4 labels of sentiment 
*  highly negative for range of sentiment 2 to 3 
*  moderate negative for range of sentiment from 4 to 5 
* moderate postive for range of sentiment from 6 to 7 
* highly positive for range of sentiment from 8 to 10 
## More better we can bring the things to 3 classes like Postive , Negative , Neutral
 since we have balance our distribution for sentiments that's why we 
*  so 2 to 4 negative , 5 to 6 neutral , 7 to 10 postive sentiment , let's see a visualization 
## After observing  the results from neural nets i have changed to binary classification as or 0 and 1 

In [ ]:
#tempdf.loc[tempdf.num_hrefs <= 4.0,'num_hrefs']=1
final_df.loc[(final_df.Sentiment <=10) & (final_df.Sentiment >=6 ),'Bins']=1
final_df.loc[(final_df.Sentiment <=5) ,'Bins']=0

In [ ]:
final_df.shape

In [ ]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import pandas as pd
from nltk.corpus import stopwords
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
def review_to_words( raw_review ):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, 'lxml').get_text() 
    
    # 2. Remove non-letters with regex
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                           
    
    # 4. Create set of stopwords
    stops = set(stopwords.words("english"))                  
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   
final_df['Text']=final_df['Text'].apply(review_to_words)

## now we almost equal distributed data

In [ ]:
final_df.shape

In [ ]:
from sklearn.model_selection import train_test_split
postive=final_df.loc[final_df['Bins'] == 1.0]
negative= final_df.loc[final_df["Bins"]== 0.0]

## Data Augumentation in NLP 
* as we are provided with less data for long articles , it is indeed that we need more data to remove overfitting because to understand long article context and semantics learning data is needed atleast of 1 lakh rows , because i have done experimentation on dataset on kaggle on dataset of bag of words meet bag of popcorn, bring more data of movie review and used modified architecture i'm having most voted kernel on that dataset  

## before augumentation and using hyper optimization we are with .7948 accuracy without overfitting at 7th epoch

## our gradient was getting diverged by small decimal at each epoch and log loss was increasing with very very small steps so at last some tuning was applied and .7948 accuracy was bring down with out any overfitting 

## but after 7th epoch overfitting was started 

## While data augmentation is increasingly being used in machine learning to train models to classify images, when it comes to natural language processing (NLP) and text analytics, its application is still limited.

## That’s partly because data augmentation is relatively new. But most importantly, it simply hasn’t been explored much in the text analytics space yet. It’s also less intuitive to do in NLP than it is with images.

## What is data augmentation?

* Let’s first consider the challenge of classifying specific images.

* Say we want our model to be able to correctly classify images of cats and dogs, but we only have a limited number of labeled   images of cats and dogs. How can we create more to better train our model? And how can we make sure our model doesn’t just learn to recognize specific cats and dogs from among the few examples that we do have?

* To create more images, we could modify slightly the existing ones. For example, we could flip an image of a dog horizontally, shift it slightly to the left or to the right, or zoom in or out. (See Exhibit 1.) We could also combine those modifications in various ways.

## Text analytics: consider the message

* Text, like a picture of a dog, cannot just be indiscriminately flipped and shifted.

* If we did indiscriminately flip and/or shift it, we would end up with examples that would confuse our model, not make it smarter. Think of trying to train a model to classify movie reviews using both the review text and the same text written in reverse. It would make no sense.

* So, while we can use data augmentation to get more text samples and improve our NLP models, we must first consider the message the text is conveying and the format in which it’s being presented.
* Let’s take an example of a document with a specific format, like a résumé.
 A résumé is usually laid out in sections, such as personal information, work experience, education, interests, hobbies,
 
* From a content perspective, those sections are independent from each other. In other words, each section would convey the same message and meaning even if the order in which they appeared in the document changed. If we shuffled the paragraphs in a résumé, we’d get another résumé conveying the same message.
* resume has no format , anyone can write the any section at any position but that will change the meaning of resume independent sections 
* so shuffling section will not change the definition of resume same as with paragraph whether last is at first position or first is at last overall sentiment of article will be negative or positive 


## As i have done research on 4 techniques to bring augumentation in nlp 

## 1st is paragraph shuffling - shuffling will be done in such way we can new articles and no repeatation will be there 

## why shuffling will not affect context or sentiment of article
   * The difference in the generated texts is based on the changed position of the sentences. As shuffling works as long as the context of the text is maintained. With sentence tokenization,the sentiment conveyed in the original text is still maintained. It would be lost if reviews were tokenized into words and then shuffled. Actually, although the meaning of a text may be lost due to words shuffling, it would not be an issue because feature extraction does consider order of words in a text.
   * Wouldn’t just tokenizing a text into sentences and then shuffling lead to over-fitting? You may ask. That’s possible to some extend. Over-fitting can occur when there is a high variance in the sentiment vocabulary used in different reviews. The two reasons why over-fitting is less likely to occur as a result of augmentation used are:
   * Based on what was observed from the negative sentiment examples that were used for augmentation, the vocabulary, or rather synonyms, does not seem to divert too much from phrases like:
        falling of price
        market closed at this price of stock 
        interest rate rising  ,
        jump to a new position 
        gold prices rose etc



## even neural nets make generalized pattern for indentifying postive or negative sentiment , shuffling the paragraph will not impose negative effect in generalization of pattern if context is not that much changed  , a pattern on which if you pass financial news article in anyway it will be able to detect that sentiment very accurately

In [ ]:
from nltk import sent_tokenize
import random
def tokenize(text):
    '''text: list of text documents'''
    tokenized =  sent_tokenize(text)
    return tokenized

def shuffle_tokenized(text):
    random.shuffle(text)
    newl=list(text)
    shuffled.append(newl)
    return text

aug_pos= []
aug_neg=[]
reps=[]
for ng_rev in postive['Text']:
    tok = tokenize(ng_rev)
    shuffled= [tok]
    #print(ng_rev)
    for i in range(17):

        shuffle_tokenized(shuffled[-1])
    for k in shuffled:
        '''create new review by joining the shuffled sentences'''
        s = ' '
        new_rev = s.join(k)
        if new_rev not in aug_pos:
            aug_pos.append(new_rev)
        else:
            reps.append(new_rev)
for ng_rev in negative['Text']:
    tok = tokenize(ng_rev)
    shuffled= [tok]
    #print(ng_rev)
    for i in range(17):

        shuffle_tokenized(shuffled[-1])
    for k in shuffled:
        '''create new review by joining the shuffled sentences'''
        s = ' '
        new_rev = s.join(k)
        if new_rev not in aug_neg:
            aug_neg.append(new_rev)
        else:
            reps.append(new_rev)
    

In [ ]:
new_positive=pd.DataFrame(data=aug_pos,columns=['Text'])
new_positive["Bins"]=1.0
new_negative=pd.DataFrame(data=aug_neg, columns=["Text"])
new_negative['Bins']=0.0

In [ ]:
new_positive.shape ,new_negative.shape

## Synonyms augumentation is a technique where similar meaning words can be replaced , we are replacing every other word except Determiners and Proper Noun or Noun 

## Antonymns augumenation is a technique where opposite meaning of words can be replaced , for example , i love you become i hate you , we can change positive to negative and negative to positive sentences 

## Unfortunatly both the techniques were not giving good results reasons : 
* if i changed only adverb then data duplication went on higher rate chances were there 
* long articles this technique will never work , it good for news headlines sentiment or shorter text but not for long one 


In [ ]:
#from nltk.corpus import wordnet
#from nltk.tokenize import word_tokenize
#from random import randint
#import nltk.data

# Load a text file if required
#def syno_aug(text):
 #   output=""
  #  new_review = []

# Load the pretrained neural net
   # tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Tokenize the text
    #tokenized = tokenizer.tokenize(text)

# Get the list of words from the entire text
    #words = word_tokenize(text)

# Identify the parts of speech
    #tagged = nltk.pos_tag(words)

    #for i in range(0,len(words)):
        #replacements = []

    # Only replace nouns with nouns, vowels with vowels etc.
     #   for syn in wordnet.synsets(words[i]):

        # Do not attempt to replace proper nouns or determiners
      #      if tagged[i][1] =='NNP' or tagged[i][1] =='DT':
        #        break
       # 
        # The tokenizer returns strings like NNP, VBP etc
        # but the wordnet synonyms has tags like .n.
        # So we extract the first character from NNP ie n
        # then we check if the dictionary word has a .n. or not 
         #   word_type = tagged[i][1][0].lower()
          #  if syn.name().find("."+word_type+"."):
            # extract the word only
           #     r = syn.name()[0:syn.name().find(".")]
            #    replacements.append(r)
        
       # if len(replacements) > 0:
            # Choose a random replacement
        #    replacement = replacements[randint(0,len(replacements)-1)]
         #   output = output + " " + replacement
            
        #else:
            # If no replacement could be found, then just use the
            # original word
         #   output = output + " " + words[i]
    #new_review.append(output)
    #return(new_review)
# Load a text file if required
#def anto_aug(text):
 #   output=""
  #  new_review = []

# Load the pretrained neural net
   # tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Tokenize the text
    #tokenized = tokenizer.tokenize(text)

# Get the list of words from the entire text
    #words = word_tokenize(text)

# Identify the parts of speech
    #tagged = nltk.pos_tag(words)

    #for i in range(0,len(words)):
     #   replacements = []

    # Only replace nouns with nouns, vowels with vowels etc.
      #  for syn in wordnet.synsets(words[i]):
       #     for lemma in syn.lemmas():
        #        for antonym in lemma.antonyms():

        # Do not attempt to replace proper nouns or determiners
         #           if tagged[i][1] =='NNP' or tagged[i][1] =='DT':
          #              break
        
        # The tokenizer returns strings like NNP, VBP etc
        # but the wordnet synonyms has tags like .n.
        # So we extract the first character from NNP ie n
        # then we check if the dictionary word has a .n. or not 
           #         word_type = tagged[i][1][0].lower()
            #        if antonym.name().find("."+word_type+"."):
            # extract the word only
             #           r = antonym.name()[0:syn.name().find(".")]
              #          replacements.append(r)
        
      #  if len(replacements) > 0:
            # Choose a random replacement
       #     replacement = replacements[randint(0,len(replacements)-1)]
        #    output = output + " " + replacement
            
        #else:
            # If no replacement could be found, then just use the
        # original word
         #   output = output + " " + words[i]
 #   new_review.append(output)
  #  return(new_review)

# web scrapper and summary generator of articles for data augumenation 

In [ ]:
#import numpy as np
#from newspaper import Article, ArticleException
#import requests
#links=[]
#def text_extractor2(link):
 #   article = Article(link)
  #  try:
   #     article.download()
    #    article.parse()
     #   article = article.text[:]
      #  text=summarize(article,ratio=0.7)
    #except (ArticleException,ValueError):
     #   print(link)
      #  links.append(link)
       # return np.nan
   # return text
    



#final_df["summary"]=final_df[15001:].URL.apply(text_extractor2)
    

In [ ]:
final_df.isnull().sum()

## importing summarize dataset with encoding latin-1 , concated them and reshuffling it 

In [ ]:
summary_df1=pd.read_csv('../input/summary-finance/summary_3314.csv',encoding='latin-1')
summary_df2=pd.read_csv('../input/summary-finance/summary_15001.csv',encoding='latin-1')
summary_df3=pd.read_csv('../input/summary-finance/summary_17191.csv',encoding='latin-1')
summary_df4=pd.read_csv('../input/summary-finance/summary_6000.csv',encoding='latin-1')

# We have also used summarization augumentation it was leading good result at some epochs but not much greater than shuffling , it was tested on local system , so i have placed code here also so anyone can study this also use it in own purpose

In [ ]:
summary=pd.concat([summary_df1,summary_df2,summary_df3,summary_df4])

In [ ]:
summary= summary.sample(frac=1).reset_index(drop=True)

In [ ]:
summary

## Cleaning of summary needs to be done because article were coming from web 

In [ ]:
#summary['summary'] = summary['summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#summary['summary']=summary['summary'].str.replace('[^\w\s]','')
#summary['Text']=summary['Text'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))
#freq = pd.Series(' '.join(summary['summary']).split()).value_counts()[:20] # removing top 20 common words
#rare = pd.Series(' '.join(summary['summary']).split()).value_counts()[-20:] #removing top 20 rare words
#freq=list(freq.index)
#rare=list(rare.index)
#summary['summary']=summary['summary'].apply(lambda x :" ".join(x for x in x.split() if x not in freq))
#summary['summary']=summary['summary'].apply(lambda x :" ".join(x for x in x.split() if x not in rare))
#summary['summary'] = summary['summary'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
#summary=summary.loc[:,['Bins','summary']]

In [ ]:
#summary=summary.rename(columns={'Bins':'Target','summary':'Text'})
new_negative=new_negative.rename(columns={'Bins':'Target','Text':'Text'})
new_positive=new_positive.rename(columns={'Bins':'Target','Text':'Text'})

In [ ]:
sentences=pd.concat([new_positive,new_negative])

## 1st we will see without augumentation what results are coming 

In [ ]:
final_new = final_df.loc[:,['Text','Bins']]
final_new=final_new.rename(columns={'Bins':'Target','Text':'Text'})
final_update=pd.concat([final_new])
final_update= final_update.sample(frac=1).reset_index(drop=True)
test_data=final_update.sample(3000)
final_update=final_update.drop(test_data.index)

In [ ]:
x = final_update.Text
y = final_update.Target.astype(int)
final_update.shape

 Background of the Techniques Convolution Neural Networks (CNN):

CNN’s are efficient for sentence classification tasks as the convolution layers can extract features horizontally from multiple words . These characteristics are essential for classification tasks as it is tricky to find clues about class memberships especially when these clues can appear in different orders in the input. CNN has also been used for document topic classifications where a single local phrase could aid in establishing the topic regardless of the position where it appears in the document. They found that CNN is powerful enough to find these local indicators due to the powerful combination of the convolution and pooling layers. Long Short-Term Memory (LSTM):

An example of LSTM’s effectiveness is its ability to capture changing sentiment in a tweet. A sentence such as “The movie was fine but not to my expectation” contains words with conflicting sentiments which is not able to be inferred accurately by a typical neural network. However, LSTM will learn that the sentiments expressed towards the end of the sentence would carry more important context compared to the words at the start.

CNN — LSTM Model:

The model architecture is . We initialized the model with Keras’ Sequential layer and added the embedding layer as the first layer. By using the embedding layer, the positive integers is turned into a dense vector of fixed size and this new representations will be passed to the CNN layer. Each filter in the CNN will detect specific features or patterns and then it will be pooled to a smaller dimension in the max-pooling layer. These features are then passed into a single LSTM layer of 100 units. Then, the LSTM outputs are then fed to a Fully Connected Layer (FCL) which was built using Keras’s Dense layer. As there are five labels to be predicted, a softmax activation function was used at the output layer.

# Current Model

In [ ]:
from IPython.display import Image
Image("../input/cnnlstm/System-architecture-of-the-proposed-regional-CNN-LSTM-model.png")

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers import MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,AveragePooling1D,MaxPooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam ,SGD
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation,GRU,Bidirectional
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Conv1D
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D,Flatten
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers.advanced_activations import LeakyReLU ,ELU 

## now we are good to proceed for model building 

In [ ]:
list_classes = ["Target"]
list_sentences_train = final_update["Text"]
list_sentences_test = test_data["Text"]
y_test=test_data["Target"]

In [ ]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [ ]:
import numpy as np
totalNumWords = [len(one_comment) for one_comment in list_tokenized_train]
plt.hist(totalNumWords,bins = np.arange(0,410,10))#[0,50,100,150,200,250,300,350,400])#,450,500,550,600,650,700,750,800,850,900])
plt.xlabel("Distribution of comment")
plt.ylabel("no of comments")
plt.title("no of comments vs no of words distribution ")
plt.show()

In [ ]:
maxlen = 400
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te=pad_sequences(list_tokenized_test,maxlen=maxlen)

In [ ]:
inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(98, activation='relu',return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = LeakyReLU(0.12)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
checkpointer = ModelCheckpoint(filepath="checkpoint.h5", verbose=1, 
                              save_best_only=False)
model.compile(loss='binary_crossentropy', optimizer='adam', 
                              metrics=['accuracy'])


In [ ]:
model.summary()

# we will be stoping this model because we know this model will never work with this much data

In [ ]:
# Start to train model 
history1 = model.fit(X_t, y, 
                    batch_size=32, 
                    epochs=1, 
                    verbose=1, 
                    validation_data=[X_te,y_test],
                    callbacks=[checkpointer],
                    shuffle=True)

# Since we can easily model is not making any improvement we have to stop the model

## Now using augumented dataset

In [ ]:
final_df.shape

In [ ]:
final_new = final_df.loc[:,['Text','Bins']]
final_new=final_new.rename(columns={'Bins':'Target','Text':'Text'})
final_update=pd.concat([final_new,new_positive,new_negative])
final_update= final_update.sample(frac=1).reset_index(drop=True)
test_data=final_update.sample(3000)
final_update=final_update.drop(test_data.index)
x = final_update.Text
y = final_update.Target.astype(int)
final_update.shape

# WILL ALWAYS BE TESTING ORIGINAL 3000 TEST CASES 

In [ ]:
list_classes = ["Target"]
list_sentences_train = final_update["Text"]
max_features = 6000
maxlen = 400
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
list_sentences_test=test_data["Text"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
y_test=test_data["Target"]

In [ ]:
inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = Conv1D(64, 3, activation='relu')(x)
x = Conv1D(128,3,activation='relu')(x)
x = AveragePooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)
x = LSTM(98, activation='relu',return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = ELU(0.2)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model1 = Model(inputs=inp, outputs=x)
checkpointer = ModelCheckpoint(filepath="checkpoint.h5", verbose=1, 
                              save_best_only=False)
model1.compile(loss='binary_crossentropy', optimizer='adam', 
                              metrics=['accuracy'])
model1.summary()

In [ ]:
# Start to train model 
history1 = model1.fit(X_t, y, 
                    batch_size=32, 
                    epochs=7, 
                    verbose=1, 
                    validation_data=[X_te,y_test],
                    callbacks=[checkpointer],
                    shuffle=True)

# So after augumentation training and testing accuracy incrased but we can see overfitting also introduced 
# .9203 accuracy is highest we can achieve from the augumentation beaten the bloomberg model, now we are going for further new approach called N-gram multi channel CNN

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# summarize history for accuracy
plt.plot(history1.history['acc'])
plt.plot(history1.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("model_accuracy_100_salinas.svg")
plt.show()

# summarize history for loss 
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("model_loss_100_salinas.svg")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
prediction = model1.predict(X_te)
y_pred = (prediction > 0.5)
print(accuracy_score(y_pred,y_test))
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

## Another approach
The model can be expanded by using multiple parallel convolutional neural networks that read the source document using different kernel sizes. This, in effect, creates a multichannel convolutional neural network for text that reads text with different n-gram sizes (groups of words).

Define Model
A standard model for document classification is to use an Embedding layer as input, followed by a one-dimensional convolutional neural network, pooling layer, and then a prediction output layer.

The kernel size in the convolutional layer defines the number of words to consider as the convolution is passed across the input text document, providing a grouping parameter.

A multi-channel convolutional neural network for document classification involves using multiple versions of the standard model with different sized kernels. This allows the document to be processed at different resolutions or different n-grams (groups of words) at a time, whilst the model learns how to best integrate these interpretations.

This approach was first described by Yoon Kim in his 2014 paper titled “Convolutional Neural Networks for Sentence Classification.”

In [ ]:
def define_model(length,vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 256)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=5, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 256)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 256)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1= Dropout(0.3)(merged)
    dense1 = Dense(50, activation='relu')(merged)  
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    return model

In [ ]:
model = define_model(400, max_features)

In [ ]:
checkpointer = ModelCheckpoint(filepath="checkpoint.h5", verbose=1, 
                              save_best_only=True)
history1=model.fit([X_t,X_t,X_t], y,batch_size=32, 
                    epochs=10, 
                    verbose=1, 
                    validation_data=([X_te,X_te,X_te],[y_test]),
                    callbacks=[checkpointer],
                    shuffle=True)


In [ ]:
# So we reached to the the highest accuracy with less log loss score 

In [ ]:
# summarize history for accuracy
plt.plot(history1.history['acc'])
plt.plot(history1.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("model_accuracy_100_salinas.svg")
plt.show()

# summarize history for loss 
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("model_loss_100_salinas.svg")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score ,classification_report

In [ ]:
preds = model.predict([X_te,X_te,X_te])
preds = (preds[:,0] > 0.5).astype(np.int)

In [ ]:
print(accuracy_score(preds,y_test))
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(preds, y_test)))
print('Confusion matrix:')
confusion_matrix(preds, y_test)

In [ ]:
def model_predict(news):
    list_sentences_train = news
    max_features = 5000
    maxlen = 400
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(list_sentences_train))
    list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
    X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
    output=model.predict([X_t,X_t,X_t])
    return output

# postive news it is  # investment in tesla in electromotive cars
news=["""nvestors interested in electric cars have a variety of options. Automakers such as Tesla Motors exclusively manufacture electric vehicles and may be directly invested in by purchasing stock. Companies within the automotive sector that manufacture vehicle parts or supply raw materials used in producing electric cars are another means of gaining portfolio exposure to electric cars. Another slightly less risky option is to invest in exchange-traded funds (ETFs) with holdings in securities related to electric vehicle production or electric vehicle parts.

Some major automakers, such as Toyota, are investing heavily in electric vehicles and allow investors to choose both traditional and electric vehicles for their investments. Chevrolet and Nissan have also made notable electric car models available in the U.S. market. Investors should carefully consider available investment opportunities and evaluate the potential risk-return tradeoff offered by electric vehicles and the automotive industry.

Many manufacturers develop auto parts for traditional and electric vehicles. Polypore International (PPO) produces lead-acid batteries used in both conventional and electric vehicles. This stock offers investors the opportunity to invest generally in the production of vehicle batteries. As electric vehicle and conventional vehicle usage grows, more batteries will be needed, and this company will likely benefit from increased global car demand.

Another battery company, Plug Power (PLUG), manufactures hydrogen fuel cell batteries used in electric vehicles and many other types of electronic equipment. These batteries may replace lead-acid batteries in fork lifts. Plug Power batteries are also used outside of the automotive industry, giving the company a large market.

Sociedad Quimica y Minera (SQM) is a major supplier of lithium, an element used in many batteries powering electric vehicles and other clean technologies. Investment in companies such as Polypore International, Plug Power, and SQM offers portfolio exposure to electric vehicles while also maintaining diverse holdings outside the automotive industry.

Electric Vehicles Exchange-Traded Funds
Exchange-traded funds that track electric vehicles are another possible opportunity for investors. These funds allow investors to purchase shares in funds that track electric vehicle industry development. Investments are spread across multiple companies, reducing investment risk and offering returns similar to the average returns of the entire sector. ETFs track gains and losses of stock indexes and are traded directly on the stock market in a means similar to stock trading. Just as in traditional stock trading, stop-loss limits may be placed, and dividends are paid to brokerage accounts.

Significant ETFs that include electric vehicle stock and supplier stock include QCLN and LIT. The First Trust NASDAQ Clean Edge Green Energy Index Fund (QCLN) has Tesla among its holdings and includes other companies with green technology offerings. Global X Lithium (LIT) tracks lithium suppliers and battery companies. This fund's most significant holdings include FMC Corporation, Avalon Rare Metals Incorporated, and Rockwood.
"""]
output=model_predict(news)
print("probability for this news",output)